In [25]:
# Importing required libraries
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
# Defining the training path of the images and the .csv file for Labels and Value
train = pd.read_csv('annotations.csv')
#test = pd.read_excel(r'C:\Users\HP\OneDrive - iitgn.ac.in\Desktop\IITD hack\SoML-50\test.xlsx')
train_folder = 'data'
#test_folder = r'C:\Users\HP\OneDrive - iitgn.ac.in\Desktop\IITD hack\SoML-50\test_imgs'

In [27]:
train.head()

,Image,Label,Value
0,1.jpg,infix,7
1,2.jpg,postfix,0
2,3.jpg,infix,21
3,4.jpg,infix,5
4,5.jpg,prefix,12


In [28]:
train['Value'] = train['Value'].astype('str')

In [29]:
train_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_dataframe(dataframe=train, directory=train_folder, x_col='Image',
                                           y_col='Label', batch_size=64, shuffle=False,
                                           class_mode='categorical', target_size=(64, 192))

Found 50000 validated image filenames belonging to 3 classes.


In [30]:
image_h, image_w = 64, 192

In [31]:
# Creating the Layers of our model with tensorflow's Sequential API
model = Sequential()

# convolutional layers
model.add(Conv2D(50, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(64, 192, 3)))

model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))

# output layer
model.add(Dense(3, activation='softmax'))

In [32]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')



In [34]:
tf.__version__


'2.8.0'

In [22]:
# training the model for 2 epochs
history = model.fit(train_data, epochs=2, steps_per_epoch=780)

Epoch 1/2
780/780 [==============================] - 40s 50ms/step - loss: 0.2021 - accuracy: 0.8944
Epoch 2/2
780/780 [==============================] - 38s 49ms/step - loss: 0.0191 - accuracy: 0.9947


In [13]:
# import pickle
# pickle.dump(model, open('model.pkl', 'wb'))
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [19]:
# saving the model as an h5 file to be used later in inference1.py and inference2.py
model.save('Model.h5')